# 891명의 데이터가 있고, 총 12개의 컬럼이 있는 걸 확인했다.

In [ ]:
import pandas as pd

passengers = pd.read_csv("passengers.csv")
print(passengers.shape)
print(passengers.head())

### 여기서 Survived가 살았는지 죽었는지 확인하는 레이블이 될 거다. 0 또는 1인데, 1이 살아남은 거다.

#  데이터 전처리하기

### 일단 생존 여부에 중요한 영향을 미쳤을 것으로 예상하는 값들만 추려보면, Sex, Age, Pcalss를 생각해볼 수 있다. 여성, 어린이, 1/2/3등석 순으로 살아남을 확률이 높다고 가설을 세워본 거다.
### 일단 머신러닝 라이브러리 sklearn에 넣어주기 전에 성별(Sex)는 숫자 데이터 1과 0으로 바꿔주자. 여성이 살아남을 확률이 높을 것 같으니 남성을 0, 여성을 1로 놓자.

In [ ]:
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})

# 결측치 채워주기
### 데이터를 보면 Age 값이 비어있는 경우가 있다. 이 경우 그냥 평균 값으로 대체해서 넣어주자.



In [ ]:
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

### Pcalss의 경우 1등석에 탔는지, 2등석에 탔는지에 대해 각각의 feature로 만들어주기 위해 컬럼을 새로 생성해보자.

In [ ]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

### 이 정도 했으면 이제 데이터 세트를 이렇게 준비할 수 있겠다.

In [ ]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

# 데이터 정규화(스케일링) 하기 ***

### 중요한 파트다. 로지스틱 회귀는 실행할 때 regularation을 사용하기 때문에 그 전에 반드시 우리가 가진 데이터를 스케일링하는 작업이 필요하다. 
### sklearn이 제공하는 StandardScaler를 활용해서 손쉽게 할 수 있다. StandardScaler는 평균 0, 표준편차 1로 변환하는 방법이지만 이외에도 최소값 0, 최대값 1이 되도록 변환하는 MinMaxScaler, 중앙값(median) 0, IQR(interquartile range) 1이 되도록 변환하는 RobustScaler 등이 있다.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

### fit_transform 은 fit과 transform을 합친 건데, fit은 일단 각 속성(feature)마다 컬럼을 만드는 작업이라고 생각하면 된다. 이후 transform을 통해 데이터를 변형시키는 거다.
### 그래서 위 코드에서도 학습세트로 fit을 한 번 해주었기 때문에, 평가세트에서는 별도로 fit을 할 필요 없이 바로 transform하면 되는 거다.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_features, train_labels)

In [ ]:
print(model.score(train_features, train_labels))

### 79% 정도 맞춘다고 한다.



In [ ]:
print(model.score(test_features, test_labels))

### 77% 정도. 나쁘지 않다.



In [ ]:
print(model.coef_)

### 이번엔 각 feature들의 계수(coefficients)를 확인해볼 차례다. 어떤 feature가 생존에 큰 영향을 주는지 확인해볼 수 있으니까.

### 아까 Sex, Age, FirstClass, SecondClass 순으로 넣었기 때문에 그 순서대로 확인해주면 된다. 성별이 1(여자)이고, 일등석 탑승 여부가 중요하다는 걸 알 수 있다. 반면 나이에 대한 계수는 음수가 나오는데 나이가 많을수록 생존 확률이 낮아진다는 의미로 이해하면 되겠다.

# 예측하기

### 이번엔 새로운 데이터를 넣어서 예측해보자.
### 예를 들면 아래와 같이 타이타닉 영화 실제 주인공이었던 Jack, Rose의 값을 임의로 만들고, 내 정보도 넣어봤다.
### (가상이니까… 1등석에 탄 걸로 해봤다.)

In [ ]:
import numpy as np

Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
ME = np.array([0.0, 32.0, 1.0, 0.0])

sample_passengers = np.array([Jack, Rose, ME])

In [ ]:
sample_passengers = scaler.transform(sample_passengers)

### Rose만 살고 Jack과 나는 죽는 걸로 나온다.



In [ ]:
print(model.predict(sample_passengers))

### 확률을 확인해보자.



In [ ]:
print(model.predict_proba(sample_passengers))

### 아깝다. 나는 살아남을 확률 44% 정도 되는 셈이다. 1등석에 타도 못 살아남다니.